In [3]:
# ------------------------------------------------------------------------------------------------ #
# Spatial Querying of GEDI Version 2 Data in R
# The following R code example demonstrates how current GEDI Finder users can migrate their GEDI 
# Version 1 spatial querying workflow from GEDI Finder to NASA's CMR in R. This tutorial will show 
# users how to use NASA's Common Metadata Repository (CMR) to perform spatial [bounding box] queries 
# for GEDI Version 2 (V2) L1B, L2A, and L2B data and how to reformat the CMR response into a list of 
# links that will allow the user to retrieve the intersecting GEDI V2 sub-orbit granules directly 
# from the LP DAAC Data Pool. The script also shows how to export the list of links to a text file.  
# ------------------------------------------------------------------------------------------------ #
# Author: Cole Krehbiel
# Last Updated: 05/10/2021
# ------------------------------------------------------------------------------------------------ #
# Check for required packages, install if not previously installed
# if ("httr" %in% rownames(installed.packages()) == FALSE) { install.packages("httr")}

# Import Packages
library(httr)

# Define Function to Query CMR
gedi_finder <- function(product, bbox) {
  
  # Define the base CMR granule search url, including LPDAAC provider name and max page size (2000 is the max allowed)
  cmr <- "https://cmr.earthdata.nasa.gov/search/granules.json?pretty=true&provider=LPDAAC_ECS&page_size=2000&concept_id="
  
  # Set up dictionary where key is GEDI shortname + version and value is CMR Concept ID
  concept_ids <- list('GEDI01_B.002'='C1908344278-LPDAAC_ECS', 
                      'GEDI02_A.002'='C1908348134-LPDAAC_ECS', 
                      'GEDI02_B.002'='C1908350066-LPDAAC_ECS')
  
  # CMR uses pagination for queries with more features returned than the page size
  page <- 1
  bbox <- sub(' ', '', bbox)  # Remove any white spaces
  granules <- list()          # Set up a list to store and append granule links to
  
  # Send GET request to CMR granule search endpoint w/ product concept ID, bbox & page number
  cmr_response <- GET(sprintf("%s%s&bounding_box=%s&pageNum=%s", cmr, concept_ids[[product]],bbox,page))
  
  # Verify the request submission was successful
  if (cmr_response$status_code==200){
    
    # Send GET request to CMR granule search endpoint w/ product concept ID, bbox & page number, format return as a list
    cmr_response <- content(GET(sprintf("%s%s&bounding_box=%s&pageNum=%s", cmr, concept_ids[[product]],bbox,page)))$feed$entry
    
    # If 2000 features are returned, move to the next page and submit another request, and append to the response
    while(length(cmr_response) %% 2000 == 0){
      page <- page + 1
      cmr_response <- c(cmr_response, content(GET(sprintf("%s%s&bounding_box=%s&pageNum=%s", cmr, concept_ids[[product]],bbox,page)))$feed$entry)
    }
    
    # CMR returns more info than just the Data Pool links, below use for loop to go through each feature, grab DP link, and add to list
    for (i in 1:length(cmr_response)) {
      granules[[i]] <- cmr_response[[i]]$links[[1]]$href
    }
    
    # Return the list of links
    return(granules)
  } else {
    
    # If the request did not complete successfully, print out the response from CMR
    print(content(GET(sprintf("%s%s&bounding_box=%s&pageNum=%s", cmr, concept_ids[[product]],bbox,page)))$errors)
  }
}

In [4]:
# User-provided inputs (UPDATE FOR YOUR DESIRED PRODUCT AND BOUNDING BOX REGION OF INTEREST)
product <- 'GEDI02_A.002'           # Options include 'GEDI01_B.002', 'GEDI02_A.002', 'GEDI02_B.002'
bbox <- '8.004,11.001,8.083,11.311'  # bounding box coordinates in LL Longitude, LL Latitude, UR Longitude, UR Latitude format

In [ ]:
# Call the gedi_finder function using the user-provided inputs
granules <- gedi_finder(product, bbox)
print(sprintf("%s %s Version 2 granules found.", length(granules), product))

# Export Results
# Set up output textfile name using the current datetime
outName <- sprintf("%s_GranuleList_%s.txt", sub('.002', '_002', product), format(Sys.time(), "%Y%m%d%H%M%S"))
print(outName)
# # Save to text file in current working directory
# write.table(granules, outName, row.names = FALSE, col.names = FALSE, quote = FALSE, sep='\n')
# print(sprintf("File containing links to intersecting %s Version 2 data has been saved to: %s/%s", product, getwd(), outName))